# Project 2 Intermediate 

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('LC_15_18.csv')

## Data Cleaning

### Columns we discarded
- 100% missing values: `id`, `member_id`, `url`, `next_pymnt_d`
- The same for all rows: `pymnt_plan`, `out_prncp`, `out_prncp_inv`, and `policy_code`
- Updated in past 2 months -- wouldn't have these values at loan origination: `num_tl_120dpd_2m`, and `num_tl_30dpd`
- Hardship-related variables, relating to borrowers who were on a hardship plan -- wouldn't know at the point of loan origination whether or not the borrower will have a hardship: `deferral_term`, `hardship_amount`, `hardship_dpd`, `hardship_end_date`, `hardship_flag`, `hardship_last_payment_amount`, `hardship_length`, `hardship_loan_status`, `hardship_payoff_balance_amount`, `hardship_reason`, `hardship_start_date`, `hardship_status`, `hardship_type`, `orig_projected_additional_accrued_interest`, `payment_plan_start_date`
- String variables provided by borrowers. LendingClub categorized them into other variables: `desc`, `title`, and `emp_title`
- Loan settlement/charged off related variables -- wouldn't know them at the time of funding: `debt_settlement_flag`, `debt_settlement_flag_date`, `settlement_amount`, `settlement_date`, `settlement_percentage`, `settlement_status`, `settlement_term`, `recoveries`, `collection_recovery_fee`
- Loan payment related variables -- wouldn't know them at the time of funding: `funded_amnt`, `funded_amnt_inv`, `last_pymnt_amnt`, `out_prncp`, `out_prncp_inv`, `total_pymnt`, `total_pymnt_inv`, `total_rec_int`, `total_rec_late_fee`, `total_rec_prncp`, `last_pymnt_d`, `last_credit_pull_d`, `disbursement_method`
- `initial_list_status`: as the values "W' and "F" are randomly assigned
- Columns relating to co-borrowers. These columns only have non-null values from issue dates of 2017-03-01 and later (or 2015-10-01 and later for 'dti_joint', 'annual_inc_joint', and 'verification_status_joint'), and have around 97% missing values, so we remove these columns: `annual_inc_joint`, `dti_joint`, `revol_bal_joint`, `sec_app_chargeoff_within_12_mths`, `sec_app_collections_12_mths_ex_med`, `sec_app_earliest_cr_line`, `sec_app_inq_last_6mths`, `sec_app_mort_acc`, `sec_app_mths_since_last_major_derog`, `sec_app_num_rev_accts`, `sec_app_open_acc`, `sec_app_open_act_il`, `sec_app_revol_util`, `verification_status_joint`, `sec_app_fico_range_low`, `sec_app_fico_range_high`
- Other: `zip_code`

In [3]:
# check number of rows and cols in the dataset
data.shape

(927931, 74)

In [4]:
# change data type
data['issue_d'] = pd.to_datetime(data['issue_d'])
data['earliest_cr_line'] = pd.to_datetime(data['earliest_cr_line'])

In [56]:
# check missing values in the dataset
def missing_values_table(data):
    '''
    generate a table that contains attributes and its corresponing number of missing values 
    and percentage of missing value
    '''
    #Number of null values by column
    mv_df = pd.DataFrame(data.isna().sum(),columns=['Count_Missing'])
    
    #Portion of null values by column
    mv_df['Pctg_Missing'] = mv_df['Count_Missing']/len(data)

    #Sort by Missing_Count
    mv_df = mv_df.sort_values('Count_Missing',ascending=False)  
    
    return mv_df

mv_df = missing_values_table(data)
mv_df.head()

,Count_Missing,Pctg_Missing
mths_since_last_record,753505,0.812027
mths_since_recent_bc_dlq,696123,0.750188
mths_since_last_major_derog,659938,0.711193
mths_since_recent_revol_delinq,600515,0.647155
mths_since_last_delinq,451425,0.486486


In [41]:
# Re-categorize emp_length into 4 categories
def process_emp_len(d):
    '''
    re_categorize the employment length in years to reduce the categories
    '''
    df = d.copy()
    # 0-1 years include null, < 1 year and 1 year
    df['emp_length'] = df['emp_length'].fillna('0-1 years')
    df['emp_length'] = np.where(df['emp_length'].isin([np.nan,None,'< 1 year','1 year']),'0-1 years',df['emp_length'])

    # 2-5 years include 2 years, 3 years, 4 years and 5 years
    df['emp_length'] = np.where(df['emp_length'].isin(['2 years','3 years','4 years', '5 years']),'2-5 years',df['emp_length'])

    # 6-9 years include 6 years, 7 years, 8 years and 9 years
    df['emp_length'] = np.where(df['emp_length'].isin(['6 years','7 years','8 years', '9 years']),'6-9 years',df['emp_length'])

    return df


# For missing values in columns that measure "months since": fill the nulls with maximum observed value +1
def process_month_since_cols(d):
    '''
    fill the null values with the maximum observed value + 1 in 'month since' related columns, 
    the borrowers who have never had delinquencies will have the largest value for number of months 
    since delinquency.
    '''
    df = d.copy()

    mon_since_cols = ['mo_sin_old_il_acct','mths_since_last_delinq','mths_since_last_major_derog',
                     'mths_since_last_record','mths_since_recent_bc_dlq','mths_since_recent_inq',
                     'mths_since_recent_revol_delinq','mo_sin_old_rev_tl_op','mths_since_recent_bc',
                     'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl']

    for i in mon_since_cols:
        df[i].fillna(df[i].max()+1, inplace=True)
    
    return df


# Change grade and sub_grade to numerical rankings (the lower the number, the less risky the loan)
def process_loan_grades(d):
    '''
    turn loan grade and sub_grade into ordinal mapping
    '''
    df = d.copy()

    # turn grade into ordinal mapping
    sorted_grades = sorted(d.grade.unique())
    grade_dict = dict(zip(sorted_grades, range(len(sorted_grades))))
    df['grade'] = df['grade'].map(grade_dict)

    # turn sub_grade into ordinal mapping
    sorted_subgrades = sorted(d.sub_grade.unique())
    subgrade_dict = dict(zip(sorted_subgrades, range(len(sorted_subgrades))))
    df['sub_grade'] = df['sub_grade'].map(subgrade_dict)

    return df


# Process the above transformation and a few more transformations
def process_loan_cols(d):
    df_processed = process_emp_len(d)
    df_processed = process_month_since_cols(df_processed)
    df_processed = process_loan_grades(df_processed)
    
    # add `credit_line_length` by compute the difference between the month the borrower's earliest reported credit line was opened
    # and the month which the loan was funded. Then discard the earliest_cr_line column.
    df_processed['credit_line_length'] = df_processed['issue_d'] - df_processed['earliest_cr_line']
    df_processed.drop(['earliest_cr_line'], axis=1)

    # process `revol_util`, `int_rate`, `credit_line_length`
    df_processed['revol_util'] = df_processed['revol_util'].apply(lambda x: x/100)
    df_processed['int_rate'] = df_processed['int_rate'].apply(lambda x: x/100)
    df_processed['credit_line_length'] = df_processed['credit_line_length'].apply(lambda x: x.days)
    
    # generate `fully_paid` (binary variable, 0 means not defaulted, 1 means fully paid) and drop `loan_status`
    df_processed['fully_paid'] = df_processed['loan_status'].map({'Fully Paid': 1, 'Charged Off': 0, 'Default': 0,
                                                                  'In Grace Period': 0, 'Late (16-30 days)': 0, 
                                                                  'Late (31-120 days)': 0})
    df_processed = df_processed.drop(columns='loan_status')
    
    return df_processed


In [60]:
# create a new pre-process dataset
data_new = process_loan_cols(data)

In [57]:
# check missing values in new dataset
mv_df = missing_values_table(data_new) # the greatest missing percentage of a feature is around 0.01

# get the list of variables that have missing values
mv_list = mv_df[mv_df['Count_Missing'] > 0].index.tolist() # all the variables that have missing values are float

# impute these missing values with its median
for var in mv_list:
    data_new[var] = data_new[var].fillna((data_new[var].median()))

In [67]:
# check the data type for each attribute
types_data = pd.DataFrame(data_new.dtypes, columns=['Types'])

# check the categorical data
cat_var = types_data[types_data['Types'] == 'object'].index.tolist()
cat_var

['term',
 'emp_length',
 'home_ownership',
 'verification_status',
 'purpose',
 'addr_state',
 'application_type',
 'issue_month']

In [10]:
# create dummy variables for categorical variables
data_new = pd.get_dummies(data_new, columns=cat_var, drop_first=True) 